In [1]:
from __future__ import absolute_import, print_function, division


from sklearn.utils import check_array


import numpy as np
from six.moves import range


def generate_random_column_samples(column):
    col_mask = np.isnan(column)
    n_missing = np.sum(col_mask)
    if n_missing == len(column):
        logging.warn("No observed values in column")
        return np.zeros_like(column)

    mean = np.nanmean(column)
    std = np.nanstd(column)

    if np.isclose(std, 0):
        return np.array([mean] * n_missing)
    else:
        return np.random.randn(n_missing) * std + mean

class Solver(object):
    def __init__(
            self,
            fill_method="zero",
             n_imputations=1,
            min_value=None,
            max_value=None,
            normalizer=None):
        self.fill_method = fill_method
        self.n_imputations = n_imputations
        self.min_value = min_value
        self.max_value = max_value
        self.normalizer = normalizer

    def __repr__(self):
        return str(self)

    def __str__(self):
        field_list = []
        for (k, v) in sorted(self.__dict__.items()):
            if v is None or isinstance(v, (float, int)):
                field_list.append("%s=%s" % (k, v))
            elif isinstance(v, str):
                field_list.append("%s='%s'" % (k, v))
        return "%s(%s)" % (
            self.__class__.__name__,
            ", ".join(field_list))

    def _check_input(self, X):
        if len(X.shape) != 2:
            raise ValueError("Expected 2d matrix, got %s array" % (X.shape,))

    def _check_missing_value_mask(self, missing):
        if not missing.any():
            warnings.simplefilter("always")
            warnings.warn("Input matrix is not missing any values")
        if missing.all():
            raise ValueError("Input matrix must have some non-missing values")

    def _fill_columns_with_fn(self, X, missing_mask, col_fn):
        for col_idx in range(X.shape[1]):
            missing_col = missing_mask[:, col_idx]
            n_missing = missing_col.sum()
            if n_missing == 0:
                continue
            col_data = X[:, col_idx]
            fill_values = col_fn(col_data)
            if np.all(np.isnan(fill_values)):
                fill_values = 0
            X[missing_col, col_idx] = fill_values

    def fill(
            self,
            X,
            missing_mask,
            fill_method=None,
            inplace=False):
        """
        Parameters
        ----------
        X : np.array
            Data array containing NaN entries
        missing_mask : np.array
            Boolean array indicating where NaN entries are
        fill_method : str
            "zero": fill missing entries with zeros
            "mean": fill with column means
            "median" : fill with column medians
            "min": fill with min value per column
            "random": fill with gaussian samples according to mean/std of column
        inplace : bool
            Modify matrix or fill a copy
        """
        X = check_array(X, force_all_finite=False)

        if not inplace:
            X = X.copy()

        if not fill_method:
            fill_method = self.fill_method

        if fill_method not in ("zero", "mean", "median", "min", "random"):
            raise ValueError("Invalid fill method: '%s'" % (fill_method))
        elif fill_method == "zero":
            # replace NaN's with 0
            X[missing_mask] = 0
        elif fill_method == "mean":
            self._fill_columns_with_fn(X, missing_mask, np.nanmean)
        elif fill_method == "median":
            self._fill_columns_with_fn(X, missing_mask, np.nanmedian)
        elif fill_method == "min":
            self._fill_columns_with_fn(X, missing_mask, np.nanmin)
        elif fill_method == "random":
            self._fill_columns_with_fn(
                X,
                missing_mask,
                col_fn=generate_random_column_samples)
        return X

    def prepare_input_data(self, X):
        """
        Check to make sure that the input matrix and its mask of missing
        values are valid. Returns X and missing mask.
        """
        X = check_array(X, force_all_finite=False)
        if X.dtype != "f" and X.dtype != "d":
            X = X.astype(float)

        self._check_input(X)
        missing_mask = np.isnan(X)
        self._check_missing_value_mask(missing_mask)
        return X, missing_mask

    def clip(self, X):
        """
        Clip values to fall within any global or column-wise min/max constraints
        """
        X = np.asarray(X)
        if self.min_value is not None:
            X[X < self.min_value] = self.min_value
        if self.max_value is not None:
            X[X > self.max_value] = self.max_value
        return X

    def project_result(self, X):
        """
        First undo normalization and then clip to the user-specified min/max
        range.
        """
        X = np.asarray(X)
        if self.normalizer is not None:
            X = self.normalizer.inverse_transform(X)
        return self.clip(X)

    def solve(self, X, missing_mask):
        """
        Given an initialized matrix X and a mask of where its missing values
        had been, return a completion of X.
        """
        raise ValueError("%s.solve not yet implemented!" % (
            self.__class__.__name__,))

    def fit_transform(self, X, y=None):
        """
        Fit the imputer and then transform input `X`
        Note: all imputations should have a `fit_transform` method,
        but only some (like IterativeImputer) also support inductive mode
        using `fit` or `fit_transform` on `X_train` and then `transform`
        on new `X_test`.
        """
        X_original, missing_mask = self.prepare_input_data(X)
        observed_mask = ~missing_mask
        X = X_original.copy()
        if self.normalizer is not None:
            X = self.normalizer.fit_transform(X)
        X_filled = self.fill(X, missing_mask, inplace=True)
        if not isinstance(X_filled, np.ndarray):
            raise TypeError(
                "Expected %s.fill() to return NumPy array but got %s" % (
                    self.__class__.__name__,
                    type(X_filled)))

        X_result = self.solve(X_filled, missing_mask)
        if not isinstance(X_result, np.ndarray):
            raise TypeError(
                "Expected %s.solve() to return NumPy array but got %s" % (
                    self.__class__.__name__,
                    type(X_result)))

        X_result = self.project_result(X=X_result)
        X_result[observed_mask] = X_original[observed_mask]
        return X_result

    
    def impute(self, X):
        X_original, missing_mask = self.prepare_input_data(X)
        observed_mask = ~missing_mask
        X = X_original.copy()
        if self.normalizer is not None:
            X = self.normalizer.fit_transform(X)
        X_filled = self.fill(X, missing_mask, inplace=True)
        if not isinstance(X_filled, np.ndarray):
            raise TypeError(
                "Expected %s.fill() to return NumPy array but got %s" % (
                    self.__class__.__name__,
                    type(X_filled)))

        X_result = self.solve(X_filled, missing_mask)
        if not isinstance(X_result, np.ndarray):
            raise TypeError(
                "Expected %s.solve() to return NumPy array but got %s" % (
                    self.__class__.__name__,
                    type(X_result)))

        X_result = self.project_result(X=X_result)
        X_result[observed_mask] = X_original[observed_mask]
        return X_result

    def multiple_imputations(self, X):
        """
        Generate multiple imputations of the same incomplete matrix
        """
        return [self.impute(X) for _ in range(self.n_imputations)]

    def complete(self, X):
        """
        Expects 2d float matrix with NaN entries signifying missing values
        Returns completed matrix without any NaNs.
        """
        imputations = self.multiple_imputations(X)
        if len(imputations) == 1:
            return imputations[0]
        else:
            return np.mean(imputations, axis=0)
    
    def fit(self, X, y=None):
        """
        Fit the imputer on input `X`.
        Note: all imputations should have a `fit_transform` method,
        but only some (like IterativeImputer) also support inductive mode
        using `fit` or `fit_transform` on `X_train` and then `transform`
        on new `X_test`.
        """
        raise ValueError(
            "%s.fit not implemented! This imputation algorithm likely "
            "doesn't support inductive mode. Only fit_transform is "
            "supported at this time." % (
                self.__class__.__name__,))

    def transform(self, X, y=None):
        """
        Transform input `X`.
        Note: all imputations should have a `fit_transform` method,
        but only some (like IterativeImputer) also support inductive mode
        using `fit` or `fit_transform` on `X_train` and then `transform`
        on new `X_test`.
        """
        raise ValueError(
            "%s.transform not implemented! This imputation algorithm likely "
            "doesn't support inductive mode. Only %s.fit_transform is "
            "supported at this time." % (
                self.__class__.__name__, self.__class__.__name__))
        
class NuclearNormMinimization(Solver):
    """
    Simple implementation of "Exact Matrix Completion via Convex Optimization"
    by Emmanuel Candes and Benjamin Recht using cvxpy.
    """

    def __init__(
            self,
            require_symmetric_solution=False,
            min_value=None,
            max_value=None,
            error_tolerance=0.0001,
            max_iters=50000,
            verbose=True):
        """
        Parameters
        ----------
        require_symmetric_solution : bool
            Add symmetry constraint to convex problem
        min_value : float
            Smallest possible imputed value
        max_value : float
            Largest possible imputed value
        error_tolerance : bool
            Degree of error allowed on reconstructed values. If omitted then
            defaults to 0.0001
        max_iters : int
            Maximum number of iterations for the convex solver
        verbose : bool
            Print debug info
        """
        Solver.__init__(
            self,
            min_value=min_value,
            max_value=max_value)
        self.require_symmetric_solution = require_symmetric_solution
        self.error_tolerance = error_tolerance
        self.max_iters = max_iters
        self.verbose = verbose

    def set_constraints(self, X, missing_mask, S, error_tolerance):
        """
        Parameters
        ----------
        X : np.array
            Data matrix with missing values filled in
        missing_mask : np.array
            Boolean array indicating where missing values were
        S : cvxpy.Variable
            Representation of solution variable
        """
        ok_mask = ~missing_mask
        masked_X = cvxpy.multiply(ok_mask, X)
        masked_S = cvxpy.multiply(ok_mask, S)
        abs_diff = cvxpy.abs(masked_S - masked_X)
        close_to_data = abs_diff <= error_tolerance
        constraints = [close_to_data]
        if self.require_symmetric_solution:
            constraints.append(S == S.T)

        if self.min_value is not None:
            constraints.append(S >= self.min_value)

        if self.max_value is not None:
            constraints.append(S <= self.max_value)

        return constraints

    def define_objective(self, m, n):
        """
        Parameters
        ----------
        m, n : int
            Dimensions that of solution matrix
        Returns the objective function and a variable representing the
        solution to the convex optimization problem.
        """
        # S is the completed matrix
        shape = (m, n)
        S = cvxpy.Variable(shape, name="S")
        norm = cvxpy.norm(S, "nuc")
        objective = cvxpy.Minimize(norm)
        return S, objective

    def solve(self, X, missing_mask):
        X = check_array(X, force_all_finite=False)

        m, n = X.shape
        S, objective = self.define_objective(m, n)
        constraints = self.set_constraints(
            X=X,
            missing_mask = missing_mask,
            S=S,
            error_tolerance=self.error_tolerance)
        problem = cvxpy.Problem(objective, constraints)
        problem.solve(
            verbose=self.verbose,
            solver=cvxpy.SCS,
            max_iters=self.max_iters,
            # use_indirect, see: https://github.com/cvxgrp/cvxpy/issues/547
            use_indirect=False)
        return S.value

In [35]:
import numpy.linalg as la
import scipy.linalg as scpla

import random

In [39]:
import cvxpy


def create_rank1_data(symmetric=False):
    """
    Returns 9x9 rank1 matrix with missing elements along diagonal
    """
    x = np.array(range(0, 9), dtype=float)
    y = np.array(random.sample(range(-1000000000, 1000000000), 9))
    XY = np.outer(x, y)
    XY_missing = XY.copy()

    # drop one entry
    XY_missing[0, 0] = np.nan    
    XY_missing[1, 1] = np.nan
    XY_missing[2, 2] = np.nan
    XY_missing[3, 3] = np.nan    
    XY_missing[4, 4] = np.nan    
    XY_missing[5, 5] = np.nan    
    XY_missing[6, 6] = np.nan    
    XY_missing[7, 7] = np.nan    
    XY_missing[8, 8] = np.nan    
    
    if not symmetric:
        return XY, XY_missing

    # make a symmetric matrix
    XYXY = XY.T.dot(XY)

    # drop one entry
    XYXY_missing = XYXY.copy()
    XYXY_missing[1, 2] = np.nan
    return XYXY, XYXY_missing

def test_rank1_convex_solver():
    XY_rank1, XY_missing_rank1 = create_rank1_data(symmetric=False)
#     print(XY_missing_rank1)
    solver = NuclearNormMinimization(max_iters=50000)
    XY_completed_rank1 = solver.fit_transform(XY_missing_rank1)
    
    return XY_completed_rank1
    
    assert abs(XY_completed_rank1[1, 2] - XY_rank1[1, 2]) < 0.01, \
        "Expected %0.4f but got %0.4f" % (
            XY_rank1[1, 2], XY_completed_rank1[1, 2])

def test_rank1_convex_completion():
    XY_rank1, XY_missing_rank1 = create_rank1_data(symmetric=False)
    XY_completed_rank1 = NuclearNormMinimization().complete(XY_missing_rank1)
    assert np.linalg.matrix_rank(XY_completed_rank1) == 2
    assert abs(XY_completed_rank1[1, 2] - XY_rank1[1, 2]) < 0.001, \
        "Expected %0.4f but got %0.4f" % (
            XY_rank1[1, 2], XY_completed_rank1[1, 2])    
    
def test_rank1_symmetric_convex_solver():
    XYXY_rank1, XYXY_missing_rank1 = create_rank1_data(symmetric=True)
    solver = NuclearNormMinimization(require_symmetric_solution=True)
    completed = solver.fit_transform(XYXY_missing_rank1)
    assert abs(completed[1, 2] - XYXY_rank1[1, 2]) < 0.01, \
        "Expected %0.4f but got %0.4f" % (
            XYXY_rank1[1, 2], completed[1, 2])
  
#     def test_nuclear_norm_minimization_with_low_rank_random_matrix():
#     solver = NuclearNormMinimization(max_iters=2000)
#     XY_completed = solver.fit_transform(XY_incomplete[:100])
#     _, missing_mae = reconstruction_error(
#         XY[:100], XY_completed, missing_mask[:100], name="NuclearNorm")
#     assert missing_mae < 0.1, "Error too high!"

if __name__ == "__main__":
#     create_rank1_data()
#     test_rank1_convex_completion()
    import sympy as sympy
    def minor(arr,i,j):
        # ith row, jth column removed
        return arr[np.array(list(range(i)) + list(range(i+1,arr.shape[0])))[:,np.newaxis],
                   np.array(list(range(j)) + list(range(j+1,arr.shape[1])))]
    
    global_saved_result_6 = []
    for i in range(0, 1000):
        output = test_rank1_convex_solver()
        saved_results = []
        for i in range(0, rows - 2):
            for j in range(0, columns - 2):
                minor_ = minor(output, i, j)
                if np.abs(la.det(minor_)) > 1e-36:
                    saved_results.append(np.abs(la.det(minor_)))
                    
        if len(saved_results) == 0:
            global_saved_result_6.append(output)
    print("Rank 6")
    print(global_saved_result_6)           
#     global_saved_result_7 = []
#     for i in range(0, 100):
#         output = test_rank1_convex_solver()
#         saved_results = []
#         for i in range(0, rows - 1):
#             for j in range(0, columns - 1):
#                 minor_ = minor(output, i, j)
#                 if np.abs(la.det(minor_)) > 1e-10:
#                     saved_results.append(np.abs(la.det(minor_)))
                    
#         if len(saved_results) == 0:
#             global_saved_result_6.append(output)            
#     print("Rank 7")
#     print(global_saved_result_7)
#     test_rank1_symmetric_convex_solver()
#     test_nuclear_norm_minimization_with_low_rank_random_matrix()

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 3.22e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -9.42e+30  2.35e+30  4.25e+30  2.62e-03 
    40| 9.24e-08  5.72e-08  3.04e-08  2.36e+10  2.36e+10  8.70e-07  1.05e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 2.60e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -1.10e+31  2.75e+30  4.97e+30  5.15e-03 
    40| 4.37e-08  4.35e-08  2.63e-09  2.77e+10  2.77e+10  3.93e-06  1.27e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 1.86e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -7.76e+30  1.94e+30  3.50e+30  6.94e-03 
    40| 2.87e-07  3.30e-07  5.66e-08  2.04e+10  2.04e+10  2.73e-06  1.44e-02 
-----------------------------------------------------------------

eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 6.34e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -9.62e+30  2.40e+30  4.34e+30  5.22e-03 
    40| 5.53e-08  6.30e-08  4.24e-10  2.48e+10  2.48e+10  1.96e-06  1.26e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 1.26e-02s
	Lin-sys: nnz in L factor: 1620, avg solve time: 5.30e-06s
	Cones: avg projection time: 1.07e-04s
	Acceleration: avg step time: 5.76e-05s
----------------------------------------------------------------------------
Error met

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 1.05e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -9.76e+30  2.44e+30  4.40e+30  4.98e-03 
    40| 5.89e-08  4.52e-08  5.76e-09  2.46e+10  2.46e+10  1.44e-06  1.29e-02 
-----------------------------------------------------------------

...

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 4.81e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -8.43e+30  2.11e+30  3.80e+30  6.78e-03 
    40| 1.25e-07  6.38e-08  1.97e-08  2.13e+10  2.13e+10  2.62e-06  1.42e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 1.61e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -8.54e+30  2.13e+30  3.85e+30  4.77e-03 
    40| 3.82e-07  3.39e-07  4.12e-07  2.22e+10  2.22e+10  7.67e-07  1.19e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 2.31e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -8.43e+30  2.11e+30  3.80e+30  7.40e-03 
    40| 6.38e-08  6.24e-08  2.44e-08  2.11e+10  2.11e+10  1.65e-06  1.48e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 1.66e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -1.07e+31  2.68e+30  4.83e+30  1.11e-02 
    40| 3.12e-08  2.90e-08  1.37e-09  2.64e+10  2.64e+10  4.32e-06  1.99e-02 
-----------------------------------------------------------------

----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 720
eps = 1.00e-04, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 333, constraints m = 495
Cones:	primal zero / dual free vars: 81
	linear vars: 243
	sd vars: 171, sd blks: 1
Setup time: 5.79e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.07e+20  1.54e+20  1.00e+00 -6.27e+30  1.57e+30  2.83e+30  7.48e-03 
    40| 6.36e-08  5.12e-08  1.02e-08  1.56e+10  1.56e+10  1.29e-06  1.49e-02 
-----------------------------------------------------------------